# Environment Setup for Vertex Model Monitoring with Tensorflow models

## Install Packages
Run `pip requirements.txt` in either (1) the notebook cell below or (2) in a notebook terminal window

> **TODO**

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# !pip install --no-cache-dir -r ./requirements.txt --user -q

## set vars

In [2]:
# naming convention for all cloud resources
VERSION        = "v1"              # TODO
PREFIX         = f'ra-vmm-{VERSION}'   # TODO

print(f"PREFIX = {PREFIX}")

PREFIX = ra-vmm-v1


### GCP project

In [3]:
# creds, PROJECT_ID = google.auth.default()
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

PROJECT_NUM              = !gcloud projects describe $PROJECT_ID --format="value(projectNumber)"
PROJECT_NUM              = PROJECT_NUM[0]

VERTEX_SA                = f'{PROJECT_NUM}-compute@developer.gserviceaccount.com'

# locations / regions for cloud resources
LOCATION                 = 'us-central1'        
REGION                   = LOCATION
BQ_LOCATION              = 'US'

print(f"PROJECT_ID       = {PROJECT_ID}")
print(f"PROJECT_NUM      = {PROJECT_NUM}")
print(f"LOCATION         = {LOCATION}")
print(f"REGION           = {REGION}")
print(f"BQ_LOCATION      = {BQ_LOCATION}")

PROJECT_ID       = hybrid-vertex
PROJECT_NUM      = 934903580331
LOCATION         = us-central1
REGION           = us-central1
BQ_LOCATION      = US


## Define Cloud Resource Names and Args

In [7]:
# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

# Location to write TF-Records for MovieLens 100K dataset
DATA_GCS_PREFIX          = "data"
DATA_PATH                = f"{BUCKET_URI}/{DATA_GCS_PREFIX}"

# container registry
REPOSITORY                = f'mm-ctp-{PREFIX}'
TRAIN_IMAGE_NAME          = f'train-ctp-{VERSION}'
REMOTE_IMAGE_NAME         = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{TRAIN_IMAGE_NAME}"

print(f"BUCKET_NAME          : {BUCKET_NAME}")
print(f"BUCKET_URI           : {BUCKET_URI}")
print(f"DATA_GCS_PREFIX      : {DATA_GCS_PREFIX}")
print(f"DATA_PATH            : {DATA_PATH}")
print(f"REPOSITORY           : {REPOSITORY}")
print(f"TRAIN_IMAGE_NAME     : {TRAIN_IMAGE_NAME}")
print(f"REMOTE_IMAGE_NAME    : {REMOTE_IMAGE_NAME}")

BUCKET_NAME          : ra-vmm-v1-hybrid-vertex-bucket
BUCKET_URI           : gs://ra-vmm-v1-hybrid-vertex-bucket
DATA_GCS_PREFIX      : data
DATA_PATH            : gs://ra-vmm-v1-hybrid-vertex-bucket/data
REPOSITORY           : mm-ctp-ra-vmm-v1
TRAIN_IMAGE_NAME     : train-ctp-v1
REMOTE_IMAGE_NAME    : us-central1-docker.pkg.dev/hybrid-vertex/mm-ctp-ra-vmm-v1/train-ctp-v1


In [8]:
# create bucket
! gsutil mb -l $REGION $BUCKET_URI

Creating gs://ra-vmm-v1-hybrid-vertex-bucket/...


In [9]:
! gsutil ls $BUCKET_URI

# Save Notebook Configuration Data

If you want to avoid having to re-enter these across notebooks

In [11]:
config = f"""
PROJECT_ID               = \"{PROJECT_ID}\"
PROJECT_NUM              = \"{PROJECT_NUM}\"
LOCATION                 = \"{LOCATION}\"

REGION                   = \"{REGION}\"
BQ_LOCATION              = \"{BQ_LOCATION}\"

VERTEX_SA                = \"{VERTEX_SA}\"

PREFIX                   = \"{PREFIX}\"
VERSION                  = \"{VERSION}\"

BUCKET_NAME              = \"{BUCKET_NAME}\"
BUCKET_URI               = \"{BUCKET_URI}\"
DATA_GCS_PREFIX          = \"{DATA_GCS_PREFIX}\"
DATA_PATH                = \"{DATA_PATH}\"

REPOSITORY               = \"{REPOSITORY}\"
TRAIN_IMAGE_NAME         = \"{TRAIN_IMAGE_NAME}\"
REMOTE_IMAGE_NAME        = \"{REMOTE_IMAGE_NAME}\"
"""
print(config)


PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"

VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

PREFIX                   = "ra-vmm-v1"
VERSION                  = "v1"

BUCKET_NAME              = "ra-vmm-v1-hybrid-vertex-bucket"
BUCKET_URI               = "gs://ra-vmm-v1-hybrid-vertex-bucket"
DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://ra-vmm-v1-hybrid-vertex-bucket/data"

REPOSITORY               = "mm-ctp-ra-vmm-v1"
TRAIN_IMAGE_NAME         = "train-ctp-v1"
REMOTE_IMAGE_NAME        = "us-central1-docker.pkg.dev/hybrid-vertex/mm-ctp-ra-vmm-v1/train-ctp-v1"



In [12]:
!echo '{config}' | gsutil cp - {BUCKET_URI}/config/notebook_env.py


Copying from <STDIN>...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              


In [13]:
!gsutil ls $BUCKET_URI

gs://ra-vmm-v1-hybrid-vertex-bucket/config/


# gitignore

In [14]:
%%writefile .gitignore
*.cpython-310.pyc
*checkpoint*
*.ipynb_checkpoints/*
# .gcloudignore
# .git
# .github
# *__pycache__
# *cpython-37.pyc
# .gitignore
# .DS_Store

Writing .gitignore
